# Building a Deep Neural Net for Sentiment Analysis on IMDb Reviews

## 1. Data collection and preprocessing
- Collect a dataset of IMDb reviews
- Preprocess the text data (tokenization, lowercasing, removing special characters, etc.)
- Split the dataset into training, validation, and test sets

## 2. **Model selection and architecture**
- Research different types of deep learning models (**RNN**, LSTM, GRU, CNN, Transformer)
- Decide on a model architecture
- Experiment with pre-trained models (BERT, GPT, RoBERTa) for fine-tuning

## 3. Model training and hyperparameter tuning
- Set up a training loop
- Use backpropagation to update the model's weights based on the loss function
- Experiment with different hyperparameters (learning rate, batch size, dropout rate, etc.) and optimization algorithms (Adam, RMSprop, etc.)
- Monitor performance on the validation set during training

## 4. Model evaluation and refinement
- Evaluate the model on the test set using relevant metrics (accuracy, F1 score, precision, recall, etc.)
- Identify areas for improvement and iterate on the model architecture, training process, or preprocessing techniques

## 5. "Extra for experts" ideas
- Handle class imbalance (oversampling, undersampling, or SMOTE)
- Experiment with different word embeddings (Word2Vec, GloVe, FastText) or contextual embeddings (ELMo, BERT)
- Explore advanced model architectures (multi-head attention, capsule networks, memory-augmented networks)
- Investigate transfer learning or multi-task learning
- Conduct error analysis to understand and address specific issues
- Develop a user interface or API for your sentiment analysis model

In [23]:
from typing import Tuple
import tokenizers
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import PackedSequence, pack_padded_sequence, pad_packed_sequence


class SimpleRNNCell(nn.Module):
    def __init__(
        self,
        input_size: int,
        hidden_size: int,
    ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        self.weight_ih = nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.weight_hh = nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.bias = nn.Parameter(torch.Tensor(hidden_size))

        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.weight_ih)
        nn.init.xavier_uniform_(self.weight_hh)
        nn.init.zeros_(self.bias)

    def forward(self, input, hidden):
        return F.tanh(
            torch.matmul(input, self.weight_ih)
            + torch.matmul(hidden, self.weight_hh)
            + self.bias
        )


class SimpleRNN(nn.Module):
    def __init__(
        self,
        input_size: int,
        hidden_size: int,
        num_layers: int,
    ):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.rnn_layers = nn.ModuleList(
            [
                SimpleRNNCell(input_size if layer == 0 else hidden_size, hidden_size)
                for layer in range(num_layers)
            ]
        )

    # type hints
    def forward(
        self,
        input: torch.Tensor, # shape (L, B, C)
        hidden: torch.Tensor = None,
        lengths: torch.Tensor = None,
    ):
        if isinstance(input, PackedSequence):
            input, lengths = pad_packed_sequence(input)
        
        if hidden is None:
            hidden = self.init_hidden(input.size(1))
        
        output = []
        for input_t in input:
            new_hidden = []
            for layer, rnn_cell in enumerate(self.rnn_layers):
                hidden[layer] = rnn_cell(input_t, hidden[layer])
                new_hidden.append(hidden[layer])
                input_t = hidden[layer]

            output.append(input_t.unsqueeze(0))

        output = torch.cat(output, dim=0)

        if lengths is not None:
            output = pack_padded_sequence(output, lengths, enforce_sorted=False)

        return output, torch.stack(new_hidden)

    def init_hidden(self, batch_size):
        return [
            torch.zeros(batch_size, self.hidden_size).to(
                self.rnn_layers[0].weight_ih.device
            )
            for _ in range(self.num_layers)
        ]


class SentimentAnalysisModel(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        emb_dim: int = 30,
        hidden_size: int = 40,
        n_rnn_layers: int = 1,
    ):
        super().__init__()
        self.emb = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=emb_dim,
        )
        self.rnn = nn.RNN(
            input_size=emb_dim,
            hidden_size=hidden_size,
            num_layers=n_rnn_layers,
        )

        # # Using custom RNN
        # self.rnn = SimpleRNN(
        #     input_size=emb_dim,
        #     hidden_size=hidden_size,
        #     num_layers=n_rnn_layers,
        # )

        self.seq = nn.Sequential(
            nn.Linear(hidden_size, 500),
            nn.ReLU(),
            nn.Linear(500, 2),
        )

    def forward(self, x: torch.Tensor, lengths: torch.Tensor):
        # x shape: (B, L)
        # convert token indices to embedding values
        x = self.emb(x)
        # x shape: (B, L, Emb dim)
        
        x = x.transpose(0, 1)

        # # # # pack the sequence
        # x = pack_padded_sequence(x, lengths, enforce_sorted=False)

        # # run the rnn, only taking the final rnn hidden state from the last layer
        # # TODO: understand the difference between the two outputs more
        _, x = self.rnn(x) #, lengths=lengths)
        # x shape: (B, n_rnn_layers, Hidden size?)


        # take only the last layer
        x = x[-1, :, :]


        # x shape: (B, Hidden size?)

        return self.seq(x)


In [24]:
# load in tokenized data
data_dict = torch.load("data/imdb_data.pt")
data = data_dict["reviews"]
labels = data_dict["labels"]
lengths = data_dict["lengths"]

# split into train and test by 80:20
training_fraction = 0.8

train_data = data[: int(len(data) * training_fraction)]
train_labels = labels[: int(len(data) * training_fraction)]
train_lengths = lengths[: int(len(data) * training_fraction)]

test_data = data[int(len(data) * training_fraction) :]
test_labels = labels[int(len(data) * training_fraction) :]
test_lengths = lengths[int(len(data) * training_fraction) :]


# load in tokenizer
tokenizer = tokenizers.Tokenizer.from_file("models/tokenizer.json")
vocab_size = tokenizer.get_vocab_size()


# Training loop

In [25]:
def calculate_accuracy(
    model: nn.Module,
    train_data: torch.Tensor,
    train_labels: torch.Tensor,
    train_lengths: torch.Tensor,
    batch_size: int,
):
    # calculate overall loss (need batching for memory reasons)
    loss = 0
    total = 0
    correct = 0

    with torch.no_grad():
        for i in range(0, len(train_data), batch_size):
            training_data_batch = train_data[i : i + batch_size]
            training_labels_batch = train_labels[i : i + batch_size]
            training_lengths_batch = train_lengths[i : i + batch_size]

            output = model(training_data_batch, lengths=training_lengths_batch)
            ## Calculate correct predictions
            _, y_predicted = torch.max(output, 1)
            total += training_labels_batch.size(0)
            correct += (y_predicted == training_labels_batch).sum().item()
            loss += nn.functional.cross_entropy(output, training_labels_batch)

    return loss, correct, total


In [27]:
import torch.optim as optim
import json
import time

# nice file name including the date to store accuracy data
date_str = time.strftime("%Y%m%d-%H%M%S")

batch_size = 512
n_epochs = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create the model
model = SentimentAnalysisModel(
    vocab_size=vocab_size,
    emb_dim=30,
    hidden_size=40,
    n_rnn_layers=5,
)
model = model.to(device)

subset = batch_size * 5

train_data = train_data.to(device)[:subset]
train_labels = train_labels.to(device)[:subset]
train_lengths = train_lengths[:subset]

test_data = test_data.to(device)[:subset]
test_labels = test_labels.to(device)[:subset]
test_lengths = test_lengths[:subset]

optimizer = optim.Adam(model.parameters(), lr=0.001)

eval_interval = 5

acc_best = 0
acc_data = []
for epoch in range(1, n_epochs):
    if epoch % eval_interval == 0:
        train_loss, train_correct, train_total = calculate_accuracy(
            model, train_data, train_labels, train_lengths, batch_size
        )
        test_loss, test_correct, test_total = calculate_accuracy(
            model, test_data, test_labels, test_lengths, batch_size
        )
        torch.save(model.state_dict(), f"models/rnn_{date_str}_latest.pt")

        test_acc = test_correct / test_total * 100
        train_acc = train_correct / train_total * 100

        if test_acc > acc_best:
            torch.save(model.state_dict(), f"models/rnn_{date_str}_best.pt")
            acc_best = test_acc

        print(
            f"UPDATE - Epoch: {epoch}, "
            f"Train Acc: {train_acc:0.2f}% (loss: {train_loss:0.2f}), "
            f"Test Acc: {test_acc: 0.2f}% (loss: {test_loss:0.2f})"
        )
        acc_data.append(
            {
                "epoch": epoch,
                "train_acc": train_acc,
                "test_acc": test_acc,
                "train_loss": train_loss.item(),
                "test_loss": test_loss.item(),
            }
        )
        with open(f"models/rnn_{date_str}_acc.json", "w") as f:
            json.dump(acc_data, f, indent=4)

    cum_loss = 0

    for i in range(0, len(train_data), batch_size):
        training_data_batch = train_data[i : i + batch_size]
        training_labels_batch = train_labels[i : i + batch_size]
        training_lengths_batch = train_lengths[i : i + batch_size]

        # forward pass
        output = model(training_data_batch, lengths=training_lengths_batch)
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        loss = nn.functional.cross_entropy(output, training_labels_batch)
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        cum_loss += loss.item()


    print(f"Epoch: {epoch}, Loss: {cum_loss:0.2f}") #, {model.rnn.rnn_layers[0].weight_hh[1, :3]}")


Epoch: 1, Loss: 3.48
Epoch: 2, Loss: 3.47
Epoch: 3, Loss: 3.47
Epoch: 4, Loss: 3.47


In [18]:
# Using a standard RNN from pytorch, it struggles to train without packing, 
# in the same way our custom RNN does!


## Loading in latest model to check accuracy

In [ ]:
# load in the latest model
loaded_model = SentimentAnalysisModel(vocab_size=vocab_size)
loaded_model.load_state_dict(torch.load("models/rnn_latest.pt"))
loaded_model.to(device)

test_data = test_data.to(device)
test_labels = test_labels.to(device)

test_stats = calculate_accuracy(
    loaded_model, test_data, test_labels, test_lengths, batch_size
)
train_stats = calculate_accuracy(
    loaded_model, train_data, train_labels, train_lengths, batch_size
)


print(
    f"Test Loss: {test_stats[0]}, Test Accuracy: {test_stats[1]/test_stats[2]*100: .2f}"
)
print(
    f"Train Loss: {train_stats[0]}, Train Accuracy: {train_stats[1]/train_stats[2]*100: .2f}"
)
